In [2]:
import pandas as pd

# Load the original data (DVC will retrieve it for you if it's not present)
df_raw = pd.read_csv("data/insurance.csv", parse_dates=['TransactionMonth'], dtype={'PostalCode': 'str'})

# Simulate a cleaning step, e.g., converting PostalCode to int for some reason
# (This is just for demonstration purposes of data versioning)
df_cleaned = df_raw.copy()
# df_cleaned['PostalCode_int'] = pd.to_numeric(df_cleaned['PostalCode'], errors='coerce').fillna(0).astype(int)

# Save the "cleaned" data as a new file
cleaned_data_path = "data/processed_insurance_v1.csv"
df_cleaned.to_csv(cleaned_data_path, index=False)
print(f"Saved processed data to {cleaned_data_path}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/insurance.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data with proper parsing
df = pd.read_csv(
    "../data/insurance.csv",
    parse_dates=['TransDate'], # <--- **CHANGE THIS TO THE EXACT NAME YOU FOUND**
    dtype={'PostalCode': 'str'}
)
# Initial inspection
print(f"Dataset dimensions: {df.shape}")
print("\nMissing values per column:")
print(df.isnull().sum().sort_values(ascending=False))

ValueError: Missing column provided to 'parse_dates': 'TransDate'

In [8]:
# Calculate Loss Ratio (Primary KPI)
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']

# Add risk flags
df['HasClaim'] = np.where(df['TotalClaims'] > 0, 1, 0)
df['ClaimSeverity'] = np.where(
    df['HasClaim'] == 1,
    df['TotalClaims'],
    np.nan
)

NameError: name 'df' is not defined

In [ ]:
plt.figure(figsize=(12,6))
df.resample('M', on='TransactionMonth')['HasClaim'].mean().plot(
    title='Monthly Claim Frequency',
    ylabel='Claim Rate'
)
plt.axhline(
    df['HasClaim'].mean(),
    color='red',
    linestyle='--',
    label='Overall Mean'
)
plt.legend()

: 

In [ ]:
# Province-level analysis
province_stats = df.groupby('Province').agg({
    'LossRatio': 'mean',
    'HasClaim': 'mean',
    'TotalPremium': 'mean'
}).sort_values('LossRatio', ascending=False)

plt.figure(figsize=(10,6))
sns.heatmap(
    province_stats.T,
    annot=True,
    fmt=".2f",
    cmap="YlOrRd"
)
plt.title("Province-Level Risk Metrics")

In [ ]:
# Top 10 highest claim vehicles
top_risky = df.groupby('Make').agg({
    'HasClaim': 'mean',
    'ClaimSeverity': 'mean'
}).nlargest(10, 'HasClaim')

sns.scatterplot(
    data=top_risky,
    x='HasClaim',
    y='ClaimSeverity',
    hue=top_risky.index,
    s=100
)
plt.title("High-Risk Vehicle Identification")

In [ ]:
# Check premium distribution
plt.figure(figsize=(10,6))
sns.histplot(
    df['TotalPremium'],
    kde=True,
    bins=50
)
plt.xlim(0, df['TotalPremium'].quantile(0.95))  # Remove extreme outliers

# QQ-Plot for normality check
import scipy.stats as stats
stats.probplot(df['TotalPremium'], plot=plt)